 # **Exploration de la source de données**

conda activate corona

In [1]:
# conda install pandas ... dans le terminal 
import pandas as pd
from datetime import date, timedelta
import os

In [2]:
# Racine des fichiers quotidiens
BASE_URL = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{}.csv' 

# Date de disponibilité des fichiers
START_DATE = date(2020, 1, 22) 
END_DATE = date(2020, 3, 12)

# Répertoire de sauvergarde des fichiers bruts 
RAWFILES_DIR = '../corona/data/raw/'
PROCESSED_DIR = '../corona/data/processed/'

# Boucle de récupération des fichiers

In [3]:
delta = END_DATE - START_DATE #as timedelta

for i in range(delta.days + 1):
    day = START_DATE + timedelta(days=i)
    day_label = day.strftime("%m-%d-%Y")
    #print(day_label)
    
    virus_df = pd.read_csv(BASE_URL.format(day_label), sep=',', parse_dates=['Last Update'])
    virus_df.to_csv(os.path.join(RAWFILES_DIR, day_label + '.csv'), index=False)
    
    #print(day)
    #print(day.strftime("%m-%d-%Y"))
    

## Constitution de la table de références lat / long

In [4]:
import glob

df_list = []

# Lecture des fichiers récupérés et sélection de ceux qui ont une lat / long
for file in glob.glob(os.path.join(RAWFILES_DIR, '*.csv')):
    virus_df = pd.read_csv(file, sep=',')
    if 'Latitude' in virus_df.columns and 'Longitude' in virus_df.columns:
        df_list.append(virus_df)

all_df = pd.concat(df_list)

# Création d'une table de références pour les lat/long
(all_df[['Province/State', 'Country/Region', 'Latitude', 'Longitude']]
 .drop_duplicates(subset=['Province/State', 'Country/Region'])
 .sort_values(by=['Country/Region', 'Province/State'])
 .to_csv(os.path.join(PROCESSED_DIR, 'lat_long_table.csv'), index=False)
)

In [5]:
all_df[['Province/State', 'Country/Region', 'Latitude', 'Longitude']].drop_duplicates()

,Province/State,Country/Region,Latitude,Longitude
0,Hubei,Mainland China,30.9756,112.2707
1,NaN,South Korea,36.0000,128.0000
2,NaN,Italy,43.0000,12.0000
3,Guangdong,Mainland China,23.3417,113.4244
4,Henan,Mainland China,33.8820,113.6140
...,...,...,...,...
194,NaN,Reunion,-21.1151,55.5364
196,NaN,Turkey,38.9637,35.2433
204,Gibraltar,United Kingdom,36.1408,-5.3536
165,NaN,Cuba,22.0000,-80.0000


In [6]:
(all_df[['Province/State', 'Country/Region', 'Latitude', 'Longitude']]
 .drop_duplicates()
 .groupby(['Province/State', 'Country/Region'])
 .count()
 .sort_values(by='Latitude', ascending=False)
)

Latitude  Longitude
Province/State       Country/Region                     
British Columbia     Canada                 2          2
Middlesex County, MA US                     2          2
 Montreal, QC        Canada                 1          1
Portland, OR         US                     1          1
Queens County, NY    US                     1          1
...                                       ...        ...
Hunan                China                  1          1
                     Mainland China         1          1
Idaho                US                     1          1
Illinois             US                     1          1
Zhejiang             Mainland China         1          1

[282 rows x 2 columns]

## Création d'une table unique 

In [7]:
data_catalog = {
    'Last Update': ['<M8[ns]'],
    'Confirmed': ['float64', 'int64'],
    'Deaths': ['float64', 'int64'],
    'Recovered': ['float64', 'int64'],
    'Latitude': ['float64'],
    'Longitude': ['float64'],
}

In [8]:
all_df

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Latitude,Longitude
0,Hubei,Mainland China,2020-03-01 10:13:19,66907,2761,31536,30.9756,112.2707
1,NaN,South Korea,2020-03-01 23:43:03,3736,17,30,36.0000,128.0000
2,NaN,Italy,2020-03-01 23:23:02,1694,34,83,43.0000,12.0000
3,Guangdong,Mainland China,2020-03-01 14:13:18,1349,7,1016,23.3417,113.4244
4,Henan,Mainland China,2020-03-01 14:13:18,1272,22,1198,33.8820,113.6140
...,...,...,...,...,...,...,...,...
213,Alaska,US,2020-03-10 02:33:04,0,0,0,61.3707,-152.4044
214,Idaho,US,2020-03-10 02:33:04,0,0,0,44.2405,-114.4788
215,Maine,US,2020-03-10 02:33:04,0,0,0,44.6939,-69.3819
216,West Virginia,US,2020-03-10 02:33:04,0,0,0,38.4912,-80.9545


In [9]:
# github.com/rvm-xx/corona/notebooks

In [10]:
df_list = []
latlong_df = pd.read_csv(os.path.join(PROCESSED_DIR, 'lat_long_table.csv'))

# Lecture des fichiers récupérés et sélection de ceux qui ont une lat / long
for file in glob.glob(os.path.join(RAWFILES_DIR, '*.csv')):
    virus_df = pd.read_csv(file, sep=',', parse_dates=['Last Update'])
    if not('Latitude' in virus_df.columns and 'Longitude' in virus_df.columns):
        virus_df = virus_df.merge(latlong_df, on=['Province/State', 'Country/Region'], how='left')
        
    for field, types in data_catalog.items():
        assert virus_df[field].dtypes in types, f"Bad type for {field} in {file}"
        
    df_list.append(virus_df.assign(source=os.path.basename(file)))

all_df = pd.concat(df_list)

# Sauvegarde de la table totale
all_df.to_csv(os.path.join(PROCESSED_DIR, 'all_data.csv'), index=False)